# Training RoBERTa for binary classification

## 0. Setup

In [ ]:
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from transformers import (
    RobertaTokenizer,
    RobertaModel,
    RobertaPreTrainedModel,
    RobertaConfig,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding
)
from sklearn.metrics import classification_report

# Paths
cache_dir = "/data/resource/huggingface/hub"
train_df = pd.read_csv("../data/processed/train-test/train_set.csv")
test_df = pd.read_csv("../data/processed/train-test/test_set.csv")
model_output_dir = "../results/model_training/roberta_binary_sdoh"

/opt/anaconda/envs/keble8263-sdoh-extraction/lib/python3.10/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


## 1. Preprocess binary labels and compute weights

In [2]:
def is_sdoh_label(label_str):
    labels = label_str.strip("<LIST>").strip("</LIST>").split(",")
    return int(not (len(labels) == 1 and labels[0] == "NoSDoH"))

train_df['binary_label'] = train_df['completion'].apply(is_sdoh_label)
test_df['binary_label'] = test_df['completion'].apply(is_sdoh_label)

In [3]:
num_pos = train_df['binary_label'].sum()
num_neg = len(train_df) - num_pos
pos_weight_val = num_neg / num_pos
print(f"[INFO] Positive samples: {num_pos}, Negative samples: {num_neg}, pos_weight: {pos_weight_val:.2f}")

[INFO] Positive samples: 228, Negative samples: 336, pos_weight: 1.47


In [4]:
class BinarySDoHDataset(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.texts = dataframe['Sentence'].tolist()
        self.labels = dataframe['binary_label'].tolist()
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=128,
            return_tensors='pt'
        )
        item = {k: v.squeeze() for k, v in encoding.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

## 2. Create model with class weights

In [5]:
class RobertaBinaryClassifierWithWeight(RobertaPreTrainedModel):
    def __init__(self, config, pos_weight):
        super().__init__(config)
        self.roberta = RobertaModel(config)
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(config.hidden_size, 1)
        self.loss_fct = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([pos_weight]))
        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, labels=None):
        outputs = self.roberta(input_ids, attention_mask=attention_mask)
        pooled_output = self.dropout(outputs.last_hidden_state[:, 0, :])  # CLS token
        logits = self.classifier(pooled_output)

        if labels is not None:
            labels = labels.unsqueeze(1)  # shape: [batch_size, 1]
            loss = self.loss_fct(logits, labels)
            return {'loss': loss, 'logits': logits}
        return {'logits': logits}

## 3. Load tokenizer and config from cache

In [19]:
# model_name = "roberta-large"
model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name, cache_dir=cache_dir, local_files_only=True)
config = RobertaConfig.from_pretrained(model_name, cache_dir=cache_dir, local_files_only=True)
model = RobertaBinaryClassifierWithWeight(config, pos_weight=pos_weight_val)
model.roberta = RobertaModel.from_pretrained(model_name, config=config, cache_dir=cache_dir, local_files_only=True)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
# # Roberta Large has 24 layers, freeze bottom 18 layers
# # Freeze bottom 18 of 24 encoder layers
# for name, param in model.roberta.named_parameters():
#     if any(f"encoder.layer.{i}." in name for i in range(18)):
#         param.requires_grad = False

# Roberta Base has 12 layers, freeze bottom 10 layers
for name, param in model.roberta.named_parameters():
    if any(f"encoder.layer.{i}." in name for i in range(10)):
        param.requires_grad = False

In [23]:
def count_parameters(model):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"[INFO] Total parameters: {total:,}")
    print(f"[INFO] Trainable parameters: {trainable:,} ({trainable / total:.2%})")

count_parameters(model)

[INFO] Total parameters: 124,646,401
[INFO] Trainable parameters: 53,767,681 (43.14%)


## 4. Training setup and train

In [30]:
train_dataset = BinarySDoHDataset(train_df, tokenizer)
test_dataset = BinarySDoHDataset(test_df, tokenizer)

training_args = TrainingArguments(
    output_dir=model_output_dir,
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=9e-5,
    num_train_epochs=10,
    logging_dir=os.path.join(model_output_dir, "logs"),
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    save_total_limit=1,
    report_to=[],
    run_name="binary_sdoh_classifier_roberta"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
)

/tmp/ipykernel_2749985/1964560415.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [31]:
trainer.train()

/opt/anaconda/envs/keble8263-sdoh-extraction/lib/python3.10/site-packages/torch/nn/parallel/data_parallel.py:37: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 0 which
    has less than 75% of the memory or cores of GPU 2. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(
/opt/anaconda/envs/keble8263-sdoh-extraction/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.477094
2,No log,0.365082
3,No log,0.328899
4,No log,0.351796
5,No log,0.370145
6,No log,0.421200
7,No log,0.380856
8,No log,0.422266
9,No log,0.388175
10,No log,0.392821


/opt/anaconda/envs/keble8263-sdoh-extraction/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/opt/anaconda/envs/keble8263-sdoh-extraction/lib/python3.10/site-packages/torch/nn/parallel/data_parallel.py:37: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 0 which
    has less than 75% of the memory or cores of GPU 2. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(
/opt/anaconda/envs/keble8263-sdoh-extraction/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/opt/anaconda/envs/keble8263-sdoh-extraction/lib/python3.10/site-pa

TrainOutput(global_step=90, training_loss=0.2176656511094835, metrics={'train_runtime': 40.2224, 'train_samples_per_second': 140.221, 'train_steps_per_second': 2.238, 'total_flos': 370983257118720.0, 'train_loss': 0.2176656511094835, 'epoch': 10.0})

## 5. Evaluate

In [32]:
outputs = trainer.predict(test_dataset)
probs = torch.sigmoid(torch.tensor(outputs.predictions)).numpy().flatten()
y_pred = (probs > 0.5).astype(int)
y_true = test_df['binary_label'].values

print("\n📊 Classification Report:")
print(classification_report(y_true, y_pred, target_names=["NoSDoH", "Any SDoH"]))

/opt/anaconda/envs/keble8263-sdoh-extraction/lib/python3.10/site-packages/torch/nn/parallel/data_parallel.py:37: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 0 which
    has less than 75% of the memory or cores of GPU 2. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(
/opt/anaconda/envs/keble8263-sdoh-extraction/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



📊 Classification Report:
              precision    recall  f1-score   support

      NoSDoH       0.92      0.85      0.88       144
    Any SDoH       0.80      0.89      0.84        99

    accuracy                           0.86       243
   macro avg       0.86      0.87      0.86       243
weighted avg       0.87      0.86      0.87       243



In [33]:
results_df = pd.DataFrame({
    "Sentence": test_df["Sentence"],
    "True Label": ["NoSDoH" if y == 0 else "Any SDoH" for y in y_true],
    "Predicted Label": ["NoSDoH" if y == 0 else "Any SDoH" for y in y_pred],
    "Prob_SDoH": probs
})
results_df.to_csv(os.path.join(model_output_dir, "binary_predictions.csv"), index=False)
print(f"\n✅ Predictions saved to {model_output_dir}/binary_predictions.csv")


✅ Predictions saved to ../results/model_training/binary_sdoh_roberta/binary_predictions.csv
